In [2]:
import numpy as np

In [1]:
!python --version

Python 3.11.5


In [ ]:
#demo for evaluation
#https://gtuckerkellogg.github.io/pairwise/demo/

In [ ]:
""" 
F(i, j) = max(
            F(i - 1, j - 1) + s(xi, yj),
            maxk=0…i-1F(k, j) - γ(i - k),
            maxk=0…j-1F(i, k) - γ(j - k),
            0)   

"""

In [37]:
scores = {
    "match" : 1,
    "mismatch" : -1,
    "gap": 0
}

def s(xi,xj):
    return scores["match"] if xi == xj else scores["mismatch"]

def match_mismatch(i,j, xi, xj, matrix):
    return matrix[i-1,j-1] + s(xi, xj)

#maxk=0…i-1F(k, j) - γ(i - k)
def γ(n):
    return n * scores["gap"]

def row_gaps_score(i,j):
    gap_scores = []
    for k in range((i)):
        gap_score = matrix[k,j] - γ(i-k)
        gap_scores.append(gap_score)
    return max(gap_scores)

#maxk=0…j-1F(i, k) - γ(j - k),
def col_gaps_score(i,j):
    gap_scores = []
    for k in range((j)):
        gap_score = matrix[i,k] - γ(j-k)
        gap_scores.append(gap_score)
    return max(gap_scores)

bottom_sequence = "GGTTACA"
top_sequence = "ATTAAACAA"
n_rows = len(bottom_sequence)+1
n_cols = len(top_sequence)+1

matrix = np.zeros((n_rows, n_cols))

for i in range(1,n_rows):
    for j in range(1,n_cols):
        print(f"i: {i}, j: {j}")
        xi = bottom_sequence[i-1]
        xj = top_sequence[j-1]
        score1 = match_mismatch(i,j, xi, xj, matrix)
        score2 = row_gaps_score(i,j)
        score3 = col_gaps_score(i,j)
        top_score = max(score1, 
                        score2, 
                        score3, 
                        0)
        matrix[i,j] = top_score
        
matrix

i: 1, j: 1
i: 1, j: 2
i: 1, j: 3
i: 1, j: 4
i: 1, j: 5
i: 1, j: 6
i: 1, j: 7
i: 1, j: 8
i: 1, j: 9
i: 2, j: 1
i: 2, j: 2
i: 2, j: 3
i: 2, j: 4
i: 2, j: 5
i: 2, j: 6
i: 2, j: 7
i: 2, j: 8
i: 2, j: 9
i: 3, j: 1
i: 3, j: 2
i: 3, j: 3
i: 3, j: 4
i: 3, j: 5
i: 3, j: 6
i: 3, j: 7
i: 3, j: 8
i: 3, j: 9
i: 4, j: 1
i: 4, j: 2
i: 4, j: 3
i: 4, j: 4
i: 4, j: 5
i: 4, j: 6
i: 4, j: 7
i: 4, j: 8
i: 4, j: 9
i: 5, j: 1
i: 5, j: 2
i: 5, j: 3
i: 5, j: 4
i: 5, j: 5
i: 5, j: 6
i: 5, j: 7
i: 5, j: 8
i: 5, j: 9
i: 6, j: 1
i: 6, j: 2
i: 6, j: 3
i: 6, j: 4
i: 6, j: 5
i: 6, j: 6
i: 6, j: 7
i: 6, j: 8
i: 6, j: 9
i: 7, j: 1
i: 7, j: 2
i: 7, j: 3
i: 7, j: 4
i: 7, j: 5
i: 7, j: 6
i: 7, j: 7
i: 7, j: 8
i: 7, j: 9


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 2., 2., 2., 2., 2., 2., 2.],
       [0., 1., 1., 2., 3., 3., 3., 3., 3., 3.],
       [0., 1., 1., 2., 3., 3., 3., 4., 4., 4.],
       [0., 1., 1., 2., 3., 4., 4., 4., 5., 5.]])

In [39]:
matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 2., 2., 2., 2., 2., 2., 2.],
       [0., 1., 1., 2., 3., 3., 3., 3., 3., 3.],
       [0., 1., 1., 2., 3., 3., 3., 4., 4., 4.],
       [0., 1., 1., 2., 3., 4., 4., 4., 5., 5.]])

In [44]:
matrix.flatten(78).reshape((n_rows, n_cols))

TypeError: order must be str, not int